In [1]:
import pandas as pd
import jieba
# !pip install jieba

import numpy as np
import seaborn as sns

import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.simplefilter('ignore')

%matplotlib inline

# !pip install openpyxl

In [2]:
company_name = pd.read_excel('./RISKLABEL_Training/2_公司实体汇总_20210414_A1.xlsx', names=['name'])

In [3]:
company_name.sample(10)

,name
466,浙江哈尔斯真空器皿股份有限公司
1563,宁波宁海农村商业银行股份有限公司
8662,眉山市东坡区益民供排水工程管理有限公司
1257,宁波荣安房地产开发有限公司
14579,河源市江东新区公用事业有限公司
10939,浙江卫星石化股份有限公司
7092,深圳能源集团股份有限公司
15190,华润深国投信托有限公司
10500,上海海得控制系统股份有限公司
6320,东莞台一盈拓科技股份有限公司


In [4]:
train_data = pd.read_excel('./RISKLABEL_Training/3_训练集汇总_20210414_A1.xlsx')

In [5]:
train_data.head()

,NEWS_BASICINFO_SID,NEWS_TITLE,ABSTRACT,CONTENT,AUTHOR,SRC_URL,SOURCE_TYPE,PUBLISH_SITE,FIRST_WEB,CHANNEL,NOTICE_DT,COMPANY_NM,LABEL,Unnamed: 13
0,ID,新闻标题,摘要,正文,作者,下载源地址,文章类型,来源,首发网站名称,网站频道,发布时间,公司名称,风险标签,NaN
1,70000001,联化科技（德州）有限公司发生爆炸 两人死亡,/,联化科技（德州）有限公司发生爆炸 两人死亡,/,/,/,/,/,/,/,联化科技（德州）有限公司,安全事故,NaN
2,70000002,2005.02.06青岛啤酒股份有限公司火灾,/,2005.02.06青岛啤酒股份有限公司火灾,/,/,/,/,/,/,/,青岛啤酒股份有限公司,安全事故,NaN
3,70000003,2004.02.14河南链鑫科技有限公司火灾,/,2004.02.14河南链鑫科技有限公司火灾,/,/,/,/,/,/,/,河南链鑫科技有限公司,安全事故,NaN
4,70000004,黑龙江：连云港振兴实业集团有限公司一煤矿火灾 21人遇难1人失踪,/,黑龙江：连云港振兴实业集团有限公司一煤矿火灾 21人遇难1人失踪,/,/,/,/,/,/,/,连云港振兴实业集团有限公司,安全事故,NaN


In [6]:
train_data.drop(0, inplace=True, axis=0)

In [7]:
train_data.head()

,NEWS_BASICINFO_SID,NEWS_TITLE,ABSTRACT,CONTENT,AUTHOR,SRC_URL,SOURCE_TYPE,PUBLISH_SITE,FIRST_WEB,CHANNEL,NOTICE_DT,COMPANY_NM,LABEL,Unnamed: 13
1,70000001,联化科技（德州）有限公司发生爆炸 两人死亡,/,联化科技（德州）有限公司发生爆炸 两人死亡,/,/,/,/,/,/,/,联化科技（德州）有限公司,安全事故,NaN
2,70000002,2005.02.06青岛啤酒股份有限公司火灾,/,2005.02.06青岛啤酒股份有限公司火灾,/,/,/,/,/,/,/,青岛啤酒股份有限公司,安全事故,NaN
3,70000003,2004.02.14河南链鑫科技有限公司火灾,/,2004.02.14河南链鑫科技有限公司火灾,/,/,/,/,/,/,/,河南链鑫科技有限公司,安全事故,NaN
4,70000004,黑龙江：连云港振兴实业集团有限公司一煤矿火灾 21人遇难1人失踪,/,黑龙江：连云港振兴实业集团有限公司一煤矿火灾 21人遇难1人失踪,/,/,/,/,/,/,/,连云港振兴实业集团有限公司,安全事故,NaN
5,70000005,连云港振兴实业集团有限公司杏花煤矿发生火灾 已发现21名遇难者遗体,/,连云港振兴实业集团有限公司杏花煤矿发生火灾 已发现21名遇难者遗体,/,/,/,/,/,/,/,连云港振兴实业集团有限公司,安全事故,NaN


### 做一些分析

In [8]:
# 看均值
company_name['name'].apply(len).mean()

13.256668936064864

In [9]:
# 看极值
company_name['name'].apply(len).max()

35

In [10]:
# 看分组
company_name[company_name['name'].apply(len) < 10]

,name
25,银亿集团有限公司
33,中路股份有限公司
63,紫光集团有限公司
82,富贵鸟股份有限公司
94,中科建设开发总公司
...,...
16138,中石化销售有限公司
16139,中石油天然气集团
16140,中铁建大桥局集团
16150,冀中能源张矿集团


In [11]:
company_name[company_name['name'].apply(len) > 20]

,name
118,中国人民财产保险股份有限公司涟水支公司高沟营销服务部
996,中国太平洋财产保险股份有限公司莱芜中心支公司
1058,拉萨经济技术开发区泰通投资合伙企业（有限合伙）
1455,中国平安财产保险股份有限公司济南第二中心支公司
1546,中华联合财产保险股份有限公司濮阳中心支公司
...,...
15908,中国人寿财产保险股份有限公司烟台市中心支公司
15912,中国人民财产保险股份有限公司莆田市分公司湄洲湾北岸经济开发区营销服务部
15920,湖南娄底农村商业银行股份有限公司湘中路支行
15922,光大证券股份有限公司乌鲁木齐民主路证券营业部


In [12]:
# 缺失值统计
for col in train_data.columns[1:-3]:
    print(col, np.mean(train_data[col].fillna('/') == '/'))

NEWS_TITLE 0.0
ABSTRACT 0.9988874625588361
CONTENT 0.0
AUTHOR 0.3464270432178006
SRC_URL 0.315875053487377
SOURCE_TYPE 0.315875053487377
PUBLISH_SITE 0.315875053487377
FIRST_WEB 0.41780059905862216
CHANNEL 0.4154899443731279
NOTICE_DT 0.315875053487377


In [13]:
train_data[train_data['AUTHOR'].fillna('/') != '/']['AUTHOR'].value_counts()

每日经济新闻       341
申万宏源证券       323
格隆汇          304
新浪财经         238
中国经济网        183
            ... 
财富人生           1
中国饲料行业资讯网      1
大英县人民法院        1
晋中法院           1
领航财经资讯网        1
Name: AUTHOR, Length: 1480, dtype: int64

In [14]:
train_data[train_data['SOURCE_TYPE'].fillna('/') != '/']['SOURCE_TYPE'].value_counts()

01    6548
06    1039
05     217
09     114
02      76
Name: SOURCE_TYPE, dtype: int64

### 数据处理

In [15]:
# 拆分有如下后缀的公司名
def replace_postfix(s):
    for x in '有限责任公司 股份有限公司 控股股份公司 总公司 有限公司 投资合伙企业 子公司 公司 集团'.split(' '):
        if s[-len(x):] == x:
            return s[:-len(x)], x
    return s, None

In [16]:
company_name['name2'] = company_name['name'].apply(replace_postfix)
company_name['name_short'] = company_name['name2'].apply(lambda x: x[0])
company_name['name_postfix'] = company_name['name2'].apply(lambda x: x[1])

In [17]:
company_name.head()

,name,name2,name_short,name_postfix
0,武汉卡伊娜化妆品有限公司,"(武汉卡伊娜化妆品, 有限公司)",武汉卡伊娜化妆品,有限公司
1,北京泰禾锦绣置业有限公司,"(北京泰禾锦绣置业, 有限公司)",北京泰禾锦绣置业,有限公司
2,长城新盛信托有限责任公司,"(长城新盛信托, 有限责任公司)",长城新盛信托,有限责任公司
3,招商银行股份有限公司,"(招商银行, 股份有限公司)",招商银行,股份有限公司
4,中国长城资产管理股份有限公司,"(中国长城资产管理, 股份有限公司)",中国长城资产管理,股份有限公司


In [18]:
company_name[company_name['name_postfix'].isnull()]

,name,name2,name_short,name_postfix
87,瑞华会计师事务所（特殊普通合伙）,"(瑞华会计师事务所（特殊普通合伙）, None)",瑞华会计师事务所（特殊普通合伙）,None
118,中国人民财产保险股份有限公司涟水支公司高沟营销服务部,"(中国人民财产保险股份有限公司涟水支公司高沟营销服务部, None)",中国人民财产保险股份有限公司涟水支公司高沟营销服务部,None
138,中国工商银行股份有限公司克拉玛依石油分行,"(中国工商银行股份有限公司克拉玛依石油分行, None)",中国工商银行股份有限公司克拉玛依石油分行,None
139,中国工商银行股份有限公司襄阳分行,"(中国工商银行股份有限公司襄阳分行, None)",中国工商银行股份有限公司襄阳分行,None
216,内黄县农村信用合作联社,"(内黄县农村信用合作联社, None)",内黄县农村信用合作联社,None
...,...,...,...,...
15923,东莞银行股份有限公司合肥分行,"(东莞银行股份有限公司合肥分行, None)",东莞银行股份有限公司合肥分行,None
15924,东吴期货有限公司泉州营业部,"(东吴期货有限公司泉州营业部, None)",东吴期货有限公司泉州营业部,None
15926,大信会计师事务所（特殊普通合伙）,"(大信会计师事务所（特殊普通合伙）, None)",大信会计师事务所（特殊普通合伙）,None
15934,衡水银行股份有限公司站前支行,"(衡水银行股份有限公司站前支行, None)",衡水银行股份有限公司站前支行,None


In [19]:
train_data = train_data[['NEWS_TITLE', 'CONTENT', 'COMPANY_NM', 'LABEL']]
train_data

,NEWS_TITLE,CONTENT,COMPANY_NM,LABEL
1,联化科技（德州）有限公司发生爆炸 两人死亡,联化科技（德州）有限公司发生爆炸 两人死亡,联化科技（德州）有限公司,安全事故
2,2005.02.06青岛啤酒股份有限公司火灾,2005.02.06青岛啤酒股份有限公司火灾,青岛啤酒股份有限公司,安全事故
3,2004.02.14河南链鑫科技有限公司火灾,2004.02.14河南链鑫科技有限公司火灾,河南链鑫科技有限公司,安全事故
4,黑龙江：连云港振兴实业集团有限公司一煤矿火灾 21人遇难1人失踪,黑龙江：连云港振兴实业集团有限公司一煤矿火灾 21人遇难1人失踪,连云港振兴实业集团有限公司,安全事故
5,连云港振兴实业集团有限公司杏花煤矿发生火灾 已发现21名遇难者遗体,连云港振兴实业集团有限公司杏花煤矿发生火灾 已发现21名遇难者遗体,连云港振兴实业集团有限公司,安全事故
...,...,...,...,...
11681,资本动态| 因未及时披露减持超5%，富邦股份实控人被深交所发监管函,"<div id=""the_content"" style=""font-size:14px;"" ...",湖北富邦科技股份有限公司,信息披露违规
11682,TCL集团更名“科技”，名副其实还是任性为之？,"<div class=""content"">\n <div class=...",中科云网科技集团股份有限公司,信息披露违规
11683,信息披露不规范，永平资源遭自律监管,"<section class=""Art-Wrap ArtMainArea"" id=""ArtH...",山东永平再生资源股份有限公司,信息披露违规
11684,玉禾田：财务数据自相矛盾或粉饰业绩 子公司存近百起劳动纠纷,"<div class=""article-content""><p><span class=""b...",玉禾田环境发展集团股份有限公司,信息披露违规


### 构建模型

In [20]:
from sklearn.preprocessing import LabelEncoder

In [21]:
train_data = train_data.sample(frac=1.0)
lbl = LabelEncoder().fit(train_data['LABEL'])
train_data['LABEL'] = lbl.transform(train_data['LABEL'])

In [22]:
train_data.sample(5)

,NEWS_TITLE,CONTENT,COMPANY_NM,LABEL
9477,华夏银行扬州分行被罚70万元：贷款资金用途管控不到位,"<div class=""article-content"">\n <p>...",华夏银行股份有限公司扬州分行,11
2894,科技创新担使命 矿冶集团举办首届开放日活动,科技创新担使命 矿冶集团举办首届开放日活动,/,5
11583,长城影视：涉嫌信披违法违规 资金紧张“内核已空”,4月13日，长城影视（002071）集合竞价跌停，现报2.09元，据了解，长城影视于日前发布...,长城影视股份有限公司,1
4555,紫光集团：未能清偿“18紫光PPN002”到期利息,"<article class=""art_box"">\n\n\t\t<!--标题_s-->\n...",紫光国际控股有限公司,2
1359,贵州永吉印务股份有限公司2020年度报告摘要,贵州永吉印务股份有限公司2020年度报告摘要,/,5


In [23]:
def strcut(s):
    seg_list = jieba.cut(s)
    return ' '.join(list(seg_list))

In [24]:
train_title = train_data['NEWS_TITLE'].apply(strcut)
train_title

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\ADMINI~1\AppData\Local\Temp\jieba.cache
Loading model cost 0.563 seconds.
Prefix dict has been built successfully.


9630    小心 ！ ST 冠福 ： 公司 及 相关 当事人 收到 《 行政处罚 决定书 》 及 《 市...
7169        天通股份 ： 控股 股东 天 通高 新 将 1758 万股 股份 解除 质押 并 再 质押
4139                 凯迪 生态 退市 留下 189.5 亿 债务 ！ 涉及 十余家 信托公司
1115                 远东 股份 子公司 中标 21.43 亿元 中国 化学 电缆 采购 项目
6518         突发 公告 ！ 这家 62 年 历史 的 钢铁 设计院 破产 重整 ， 招募 投资人 ！
                              ...                        
5267    财信 信托 2019 年 净利润 下滑 74%   ， 重大 未决 诉讼 标的 金额 20....
8278                                          价外 加价 捆绑 销售
1996                      宿迁市 交通 产业 集团 有限公司 人才 招聘 的 招聘 程序
3484         深圳湾 科技 发展 有限公司 董事长 邱文 ： 科技园区 运营 的 核心 是 打造 产业
3027              招贤纳士   虚位以待 — — 中国 文化传媒 集团 2021 年 招聘 公告
Name: NEWS_TITLE, Length: 11685, dtype: object

In [25]:
train_data['LABEL'].value_counts()

5     3042
4      811
12     800
1      800
8      799
0      799
9      799
2      798
11     796
7      796
10     796
3      326
6      323
Name: LABEL, dtype: int64

### TFIDF

In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [27]:
tfidf = TfidfVectorizer(ngram_range=(1, 1))
train_title_tfidf = tfidf.fit_transform(train_title)

In [28]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import RidgeClassifier

In [29]:
clf = RidgeClassifier()

In [30]:
from sklearn.model_selection import train_test_split

In [31]:
tr_x, val_x, tr_tfidf, val_tfidf, tr_y, val_y, = train_test_split(train_data['NEWS_TITLE'], 
                                                                  train_title_tfidf, 
                                                                  train_data['LABEL'],
                                                                  stratify=train_data['LABEL'],
                                                                  test_size=0.2)

# stratify: 保持测试集与整个数据集里label的数据分类比例一致

In [32]:
clf.fit(tr_tfidf, tr_y)
clf.score(val_tfidf, val_y)

0.8737697903294822

### BERT分类

In [33]:
from transformers import BertTokenizer

In [34]:
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')
train_encoding = tokenizer(list(tr_x), truncation=True, padding=True, max_length=128)
val_encoding = tokenizer(list(val_x), truncation=True, padding=True, max_length=128)

In [35]:
train_encoding.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [36]:
from torch.utils.data import Dataset, DataLoader, TensorDataset
import torch

In [37]:
class TextDataset(Dataset):
    
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
        
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels.iloc[idx])
        return item
    
    def __len__(self):
        return len(self.labels)

In [38]:
train_dataset = TextDataset(train_encoding, tr_y)
test_dataset = TextDataset(val_encoding, val_y)

In [39]:
text = '中国人民财产保险股份有限公司涟水支公司高沟营销服务部'
tokenizer(text)
# token_type_ids区分上下句的编码

{'input_ids': [101, 704, 1744, 782, 3696, 6568, 772, 924, 7372, 5500, 819, 3300, 7361, 1062, 1385, 3878, 3717, 3118, 1062, 1385, 7770, 3765, 5852, 7218, 3302, 1218, 6956, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [40]:
text = '哈喽哈哈地方'
tokenizer(text)

{'input_ids': [101, 1506, 1617, 1506, 1506, 1765, 3175, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}

In [41]:
from transformers import BertForSequenceClassification

In [42]:
model = BertForSequenceClassification.from_pretrained('bert-base-chinese', num_labels=13)

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [43]:
train_data.shape

(11685, 4)

In [44]:
tr_tfidf.shape

(9348, 16871)

In [45]:
from transformers import AutoModelForSequenceClassification, AdamW, get_linear_schedule_with_warmup

In [46]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=True)

optim = AdamW(model.parameters(), lr=5e-5)
total_steps = len(train_loader) * 1
loss_function = torch.nn.CrossEntropyLoss()
scheduler = get_linear_schedule_with_warmup(optim, num_warmup_steps=0, num_training_steps=total_steps)

In [47]:
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [48]:
def train():
    model.train()
    total_train_loss = 0
    iter_num = 0
    total_iter = len(train_loader)
    for batch in train_loader:
        optim.zero_grad()
        input_ids = batch['input_ids'].to(device).to(torch.int64)
        attention_mask = batch['attention_mask'].to(device).to(torch.int64)
        labels = batch['labels'].to(device).to(torch.int64)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        
        loss = loss_function(outputs[1], labels)
         
        total_train_loss += loss.item()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optim.step()
        scheduler.step()
        
        iter_num += 1
        if iter_num % 100 == 0:
            print(f'epoch {epoch}, iter_num: {iter_num}, loss: {loss.item():.4f}, {iter_num / total_iter * 100:.2f}%')
    
    print(f'Epoch: {epoch}, Average training loss: {total_train_loss / len(train_loader):.4f}')

In [49]:
def validation():
    model.eval()
    total_eval_accuracy = 0
    total_eval_loss = 0
    for batch in test_loader:
        with torch.no_grad():
            input_ids = batch['input_ids'].to(device).to(torch.int64)
            attention_mask = batch['attention_mask'].to(device).to(torch.int64)
            labels = batch['labels'].to(device).to(torch.int64)
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs[0]
        logits = outputs[1]

        total_eval_loss += loss.item()
        logits = logits.detach().cpu().numpy()
        label_ids = labels.to('cpu').numpy()
        total_eval_accuracy += flat_accuracy(logits, label_ids)
        
    avg_val_accuracy = total_eval_accuracy / len(test_loader)
    print(f'Accuracy: {avg_val_accuracy:.4f}')
    print(f'Average testing loss: {total_eval_loss/len(test_loader):.4f}')
    print("-------------------------------")
    

In [50]:
for epoch in range(4):
    print('-------------------- Epoch {epoch} -----------------------')
    train()
    validation()

-------------------- Epoch {epoch} -----------------------
epoch 0, iter_num: 100, loss: 0.7604, 17.09%
epoch 0, iter_num: 200, loss: 1.1768, 34.19%
epoch 0, iter_num: 300, loss: 0.3559, 51.28%
epoch 0, iter_num: 400, loss: 0.7011, 68.38%
epoch 0, iter_num: 500, loss: 0.3889, 85.47%
Epoch: 0, Average training loss: 0.7411
Accuracy: 0.8452
Average testing loss: 0.4925
-------------------------------
-------------------- Epoch {epoch} -----------------------
epoch 1, iter_num: 100, loss: 0.2978, 17.09%
epoch 1, iter_num: 200, loss: 0.2125, 34.19%
epoch 1, iter_num: 300, loss: 0.3809, 51.28%
epoch 1, iter_num: 400, loss: 0.2053, 68.38%
epoch 1, iter_num: 500, loss: 0.1340, 85.47%
Epoch: 1, Average training loss: 0.4141
Accuracy: 0.8452
Average testing loss: 0.4951
-------------------------------
-------------------- Epoch {epoch} -----------------------
epoch 2, iter_num: 100, loss: 0.3046, 17.09%
epoch 2, iter_num: 200, loss: 0.7119, 34.19%
epoch 2, iter_num: 300, loss: 0.2639, 51.28%
ep

### 构建模型：公司主体识别

In [51]:
for row in train_data.iloc[:100].iterrows():
    match1 = company_name[company_name['name'].apply(lambda x: x in row[1].NEWS_TITLE)]
    if match1.shape[0] > 0:
        match1.loc[:, 'name_len'] = match1['name'].apply(len)
        match1 = match1.sort_values(by='name_len')
        match1 = match1.iloc[-1]['name']
    else:
        match1 = ''
    
    match2 = company_name[company_name['name_short'].apply(lambda x: x in row[1].NEWS_TITLE)]
    if match2.shape[0] > 0 and match1 == '':
        match2.loc[:, 'name_len'] = match2['name_short'].apply(len)
        match2 = match2.sort_values(by='name_len')
        match2 = match2.iloc[-1]['name']
    else:
        match2 = ''
        
    print('标题：', row[1].NEWS_TITLE)
    print('主体标签：', row[1]['COMPANY_NM'])
    print('主体识别结果：', match1 + match2)
    print('')
        

标题： 小心！ST冠福：公司及相关当事人收到《行政处罚决定书》及《市场禁入决定书》
主体标签： 冠福控股股份有限公司
主体识别结果： 

标题： 天通股份：控股股东天通高新将1758万股股份解除质押并再质押
主体标签： 天通高新集团有限公司
主体识别结果： 

标题： 凯迪生态退市留下189.5亿债务！涉及十余家信托公司
主体标签： 凯迪生态环境科技股份有限公司
主体识别结果： 

标题： 远东股份子公司中标21.43亿元中国化学电缆采购项目
主体标签： /
主体识别结果： 

标题： 突发公告！这家62年历史的钢铁设计院破产重整，招募投资人！
主体标签： 江苏省冶金设计院有限公司
主体识别结果： 

标题： 2016年云南白药集团股份有限公司公司债券（第一期）发行结果公告
主体标签： /
主体识别结果： 云南白药集团股份有限公司

标题： 2元的牙膏骗了68万余人 “通和商城”特大传销案二审宣判
主体标签： 广东雅姿精化有限公司
主体识别结果： 

标题： 洽洽食品疑信披违规：管理层称某产品销售额今年翻番，股价三连涨未见公告披露
主体标签： 洽洽食品股份有限公司
主体识别结果： 洽洽食品股份有限公司

标题： 安徽马鞍山屠宰场违规生产后续
主体标签： 马鞍山市苏润食品有限公司
主体识别结果： 

标题： 叶文钦短线交易宏昌电子 被给予警告并罚10万元
主体标签： 宏昌电子材料股份有限公司
主体识别结果： 

标题： 歌华有线：公司实际控制人由北京广播电视台变更为国务院
主体标签： 北京歌华有线电视网络股份有限公司
主体识别结果： 

标题： ST威龙自认违规 投资者可预备索赔登记
主体标签： 威龙葡萄酒股份有限公司
主体识别结果： 

标题： 2019年Q4巨亏28.6亿元 玖富“断臂”转型不利与人保对簿公堂
主体标签： 玖富数科科技集团有限责任公司
主体识别结果： 

标题： 孙珅会见中国纳豆集团股份有限公司客人
主体标签： /
主体识别结果： 

标题： 富昌证券因涉及卖空交易的内部监控缺失等遭香港证监会谴责及罚款360万元
主体标签： 富昌证券有限公司
主体识别结果： 富昌证券有限公司

标题： 黑龙江北大荒农业股份有限公司公告（系列）
主体标签： /
主体识别结果： 黑龙江北大荒农业股份有限公司

标题： ST富控濒临退市边缘 “中技系”全面败退
主体标签